# Experiment: _Combined text-/graph features vs. text-only features_

In [ ]:
from notebook_prelude import *

In [ ]:
EXPERIMENT_NAME = 'graph_combined'
experiment_data = experiment_helper.get_experiment_config_for(EXPERIMENT_NAME)
param_grid = experiment_data['params_per_type']
df = results_helper.get_results(filter_out_experiment=EXPERIMENT_NAME, filter_out_non_complete_datasets=False).fillna('(MISSING)')
pipeline_helper.remove_complex_types(pipeline_helper.flatten_nested_params(param_grid))

In [ ]:
df.groupby(['type', 'text__vectorizer', 'combined']).mean_test_f1_macro.max().to_frame()

In [ ]:
a = sklearn.feature_extraction.text.TfidfTransformer
b = a()
isinstance(None, type)